# ADX: KQL Union Query Generation 

In [ ]:
import pandas as pd

In [ ]:
# test data
df = pd.DataFrame(
    {
        'Cluster':      ["cluster_a",   "cluster_a",     "cluster_a",      "cluster_a"], 
        'Table':        ["table_a",     "table_b",       "table_c",        "table_d"], 
        'Timestamp':    ["timestamp",   "created_time",  "timestamp_col",  "start_time"],
        'Username':     ["user",        "username",      pd.NA,            "user"],
        'Hostname':     [pd.NA,         pd.NA,           "hostname",       "host"],
        'Application':  [pd.NA,         pd.NA,           "app",            "app_name"],
        'IP':           [pd.NA,         "client_ip",     "clientip",       "ip"]
    }
)
group_columns = df.columns.tolist()
group_columns.remove('Cluster')
group_columns.remove('Table')
group_columns.remove('Timestamp')

In [ ]:
# genereate KQL query string 
query_string = "union (\n"
for each_row in df.itertuples():
    query_string += f"\t{each_row.Table}\n"
    # group columns by timestamp
    query_string += f"\t| sumarize "
    for each_column in group_columns:
        each_value = each_row._asdict()[each_column]
        if each_value is not pd.NA:
            query_string += f"make_set({each_value}), "
    query_string = query_string[:-2]
    query_string += f" by bin({each_row.Timestamp}, 1hr)\n"
    # table source column
    query_string += f'\t| extend Table="{each_row.Table}"\n'
    # rename columns to consistent names 
    query_string += f"\t| project-rename Timestamp={each_row.Timestamp}, "
    for each_column in group_columns:        
        each_value = each_row._asdict()[each_column]
        if each_value is not pd.NA:
            query_string += f"{each_column}=set_{each_value}, "
    query_string = query_string[:-2]
    query_string += "\n), (\n"
query_string = query_string[:-4] +"\n"
query_string += f"| where isnotempty(Timestamp)\n"
query_string += "| project-reorder Timestamp, Table\n"
query_string += f"| sort by Timestamp\n"

In [ ]:
print(query_string)